<h1 style="color:rgb(0,120,170)">344.038, KV Multimedia Search and Retrieval (WS2023/24)</h1>
<h2 style="color:rgb(0,120,170)">Task 3_Group B</h2>

| First Name | Family Name  | Matr.Nr   |
|:-----------|:-------------|:----------|
| Harald     | Eibensteiner | K01300179 |
| Hadi       | Sanaei       | K11733444 |
| Lukas      | Troyer       | K12006666 |
| Lukas      | Wagner       | K01357626 |
| Branko     | Paunović     | K12046370 |

# Retrieval systems

#### Import datasets, setup helper interfaces

In [2]:
import pandas as pd

import numpy as np
np.random.seed(42)

from song import songs
from datasets import datasets
from retrieval import Retrieval

/home/hd/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [3]:
# # precompute retrievals (for frontend and chached lookup)

# # only do this if you want to recompute the retrieval

# retrieval = Retrieval(n=100)
# retrieval.precompute_all(threads=4)

#### Prompt for user input

In [4]:
# Number of similar songs to retrieve
num_top_similar = int(input("Enter the number of top similar tracks: "))

# Example:
#   - Title: Letterman
#   - Artist: Wiz Khalifa
query, query_row_id = songs.prompt_song_query()

Enter the number of top similar tracks:  10
Enter the song title:  Letterman
Enter the artist name:  Wiz Khalifa


## Video-Based(\<similarity\>, \<feature\>)
### 1. Video-based(similarity, incp)

In [5]:
retN = Retrieval(n=num_top_similar)
Retrieval.create_df_from_tracks(retN.top_similar_tracks(query_row_id, dataset=datasets.resnet))

Error syncing cache with disk: Expecting ',' delimiter: line 1 column 37937813 (char 37937812)


,id,similarity,song,artist,album_name
0,GY25og83Ng5O9s75,0.783461,Remind Me,Röyksopp,Melody A.M.
1,gw9zZ3xtxoEjmYeB,0.777114,Locos,Cypress Hill,Elephants on Acid
2,8syTodUYnWUZtcIo,0.771750,Voodoo Child (Slight Return),Jimi Hendrix,Electric Ladyland
3,Dtf7ZV2mJSpyuvdD,0.770228,Hollow Man,R.E.M.,Accelerate
4,VSfGMu47DQFgMW4m,0.764065,Love Is The Drug,Grace Jones,Island Life
5,WgtykoE4fvPTAvxW,0.757802,White Noise,FEMM,Femm-Isation
6,5i9Rj9XyqhEfSegl,0.755106,Barbie Tingz,Nicki Minaj,Barbie Tingz
7,fVcJKhhd7UDpaEVJ,0.754682,To Tylko Tango,Maanam,The Best Of Kora & Maanam Volume 1
8,SXz8wEnJvIm7EQVU,0.754580,Volte Para O Seu Lar,Marisa Monte,Mais
9,x88qsR3ew98Unhry,0.753878,Lost In The Moment,We Came As Romans,Cold Like War


## Fusion-Based (\<similarity\>, \<feature\>)
### Early Fusion (musicnn & resnet)

In [6]:
import pandas as pd

def early_fusion_dataframe(df1, df2):
    if len(df1) != len(df2):
        raise ValueError("Both dataframes must have the same number of rows")

    # Concatenate dataframes along columns (axis=1)
    fused_df = pd.concat([df1, df2], axis=1)

    return fused_df


df_music = pd.read_csv('datasets/id_musicnn_mmsr.tsv', sep='\t')
df_resnet = pd.read_csv('datasets/id_resnet_mmsr.tsv', sep='\t')


df_earlyf = early_fusion_dataframe(df_music, df_resnet)
#df_earlyf

df_earlyf.to_csv("datasets/id_early_fusion_mmsr.tsv", sep='\t', index=False)



In [7]:
retN = Retrieval(n=num_top_similar)
Retrieval.create_df_from_tracks(retN.top_similar_tracks(query_row_id, dataset="early_fusion"))

Error syncing cache with disk: Expecting ',' delimiter: line 1 column 37937813 (char 37937812)


AttributeError: 'str' object has no attribute 'name'

### Late Fusion (bert & tf-idf)

In [8]:
# Perform tf_idf retrieval
df_tfidf = Retrieval.create_df_from_tracks(Retrieval(n=100).top_similar_tracks(query_row_id,datasets.tf_idf))

# Perform bert retrieval
df_bert = Retrieval.create_df_from_tracks(Retrieval(n=100).top_similar_tracks(query_row_id,datasets.lyrics_bert))



Error syncing cache with disk: Expecting ',' delimiter: line 1 column 37937813 (char 37937812)
Error syncing cache with disk: Expecting ',' delimiter: line 1 column 37937813 (char 37937812)


In [9]:

# Late fusion - Score Aggregation for two DataFrames
def late_fusion_df(df_tfidf, df_bert, tfidf_weight=0.5, bert_weight=0.5):
    
    # Rename the 'similarity' column
    df_tfidf = df_tfidf.rename(columns={'similarity': 'similarity_tfidf'})
    df_bert = df_bert.rename(columns={'similarity': 'similarity_bert'})

    # Merge DataFrames on 'id'
    merged_df = pd.merge(df_bert[['id', 'similarity_bert']],df_tfidf, on='id')

    # Assuming the scores are already normalized between 0 and 1
    normalized_tfidf_scores = merged_df['similarity_tfidf'].values
    normalized_bert_scores = merged_df['similarity_bert'].values

    # Aggregate scores
    aggregated_scores = tfidf_weight * normalized_tfidf_scores + bert_weight * normalized_bert_scores

    # Add the aggregated scores to the DataFrame
    merged_df['aggregated_score'] = aggregated_scores


    # Sort DataFrame based on aggregated scores
    sorted_fusion_df = merged_df.sort_values(by='aggregated_score', ascending=False)

    return sorted_fusion_df

# Example usage
tfidf_weight = 0.6
bert_weight = 0.4
fusion_results_df = late_fusion_df(df_tfidf, df_bert, tfidf_weight, bert_weight)
fusion_results_df


,id,similarity_bert,similarity_tfidf,song,artist,album_name,aggregated_score
0,iHRt8u88DH1Dq7SP,0.698288,0.236507,Loyal,Chris Brown,X (Expanded Edition),0.421219
1,dVLg34HafHN2B9lW,0.684422,0.206531,Candy Paint,Post Malone,beerbongs & bentleys,0.397687
2,CfCJl3HWjZCvaTv3,0.667431,0.215901,Let Me Blow Ya Mind,Eve,Scorpion,0.396513
4,p6ZwetqySJuSXWdL,0.651848,0.224869,It Girl,Jason Derülo,Future History (Deluxe Edition),0.395661
3,BvavHfrEqUNNphJD,0.654582,0.195008,Believe,Eminem,Revival,0.378837
5,ZmXHCROopvjj4Ff0,0.651441,0.195305,Under The Weather,Mac Miller,Blue Slide Park,0.377760


# Evaluation 

In [10]:
# Obtain necessary information and store for easy interop
from genres import Genres

genres = Genres()

  --> No pickled data file found. Performing on-the-fly data computation and saving to 'pickled_state/precomputed_genres.pickle' for later (re-)use.
Found 10094 songs with genre information


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


AttributeError: 'FloatProgress' object has no attribute 'style'

## Accuracy
#### 1. Precision@k & Recall@k

In [ ]:
from precision_recall import PrecisionRecall

pr = PrecisionRecall(genres)
pr.compute()

In [ ]:
pr.plot_each()

In [ ]:
pr.plot_all_single()

#### 2. nDCG@10

In [ ]:
from ndcg import Ndcg

ndcg = Ndcg(genres)
ndcg.compute()
ndcg.plot()

## Beyond Accuracy
### 1. Genre Coverage@10

In [ ]:
from genre_coverage import GenreCoverage

genres_coverage = GenreCoverage(genres)
genres_coverage.compute()
genres_coverage.plot()

### 2. Genre Diversity@10

In [ ]:
from genre_diversity import GenreDiversity

genres_diversity = GenreDiversity(genres)
genres_diversity.compute()
genres_diversity.plot()

# Frontend

In [ ]:
from utils import write_song_df_to_json_file

write_song_df_to_json_file("frontend/static/songMeta.json", datasets.information.df, datasets.url.df, datasets.genres.df)

# frontend uses frontend/static/songMeta.json and retrievals/*.json
